# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `<30>`
* Students: `<Joachim Tapparel, Tim Tuuva, Lucas Biotto, Anael Buchegger>`
* Dataset: `<Free Music Archive>`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_FOLDER = '../data/'
METADATA_FOLDER =  DATA_FOLDER + 'fma_metadata/'

## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

**Il n'y a qu'une colonne label, en faite c'est le genre**

In [3]:
raw_albums = pd.read_csv(METADATA_FOLDER + 'raw_albums.csv', sep=',', encoding='utf-8', engine='python')
raw_artists = pd.read_csv(METADATA_FOLDER + 'raw_artists.csv', sep=',', encoding='utf-8', engine='python')
raw_echonest = pd.read_csv(METADATA_FOLDER + 'raw_echonest.csv', sep=',', encoding='utf-8', engine='python')
raw_genres = pd.read_csv(METADATA_FOLDER + 'raw_genres.csv', sep=',', encoding='utf-8', engine='python')
raw_tracks = pd.read_csv(METADATA_FOLDER + 'raw_tracks.csv', sep=',', encoding='utf-8', engine='python')

In [30]:
tracks = pd.read_csv(METADATA_FOLDER + 'tracks.csv',header=[0,1,2])
echonest = pd.read_csv(METADATA_FOLDER + 'echonest.csv',header=[0,1,2,3])
features = pd.read_csv(METADATA_FOLDER + 'features.csv',header=[0,1,2,3])
genres = pd.read_csv(METADATA_FOLDER + 'genres.csv')

In [41]:
# Display test
tracks.head(2)

Unnamed: 0_level_0              album                       \
  Unnamed: 0_level_1           comments         date_created   
            track_id Unnamed: 1_level_2   Unnamed: 2_level_2   
0                  2                  0  2008-11-26 01:44:45   
1                  3                  0  2008-11-26 01:44:45   

                                                              \
         date_released           engineer          favorites   
    Unnamed: 3_level_2 Unnamed: 4_level_2 Unnamed: 5_level_2   
0  2009-01-05 00:00:00                NaN                  4   
1  2009-01-05 00:00:00                NaN                  4   

                                                                               \
                  id        information            listens           producer   
  Unnamed: 6_level_2 Unnamed: 7_level_2 Unnamed: 8_level_2 Unnamed: 9_level_2   
0                  1            <p></p>               6073                NaN   
1                  1            <p></p>               6073                NaN   

          ...                       track                      \
          ...                 information            interest   
          ...         Unnamed: 43_level_2 Unnamed: 44_level_2   
0         ...                         NaN                4656   
1         ...                         NaN                1470   

                                                                          \
        language_code                                            license   
  Unnamed: 45_level_2                                Unnamed: 46_level_2   
0                  en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
1                  en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   

                                                               \
              listens            lyricist              number   
  Unnamed: 47_level_2 Unnamed: 48_level_2 Unnamed: 49_level_2   
0                1293                 NaN                   3   
1                 514                 NaN                   4   

                                                               
            publisher                tags               title  
  Unnamed: 50_level_2 Unnamed: 51_level_2 Unnamed: 52_level_2  
0                 NaN                  []                Food  
1                 NaN                  []        Electric Ave  

[2 rows x 53 columns]

In [229]:
# Copy & removing useless columns labels in level 2
tracks_cleaned = tracks.copy()
tracks_cleaned.columns = tracks_cleaned.columns.droplevel(level=2)

# TODO BEURKKK pas bien xD
# Need to create a dictionnary to rename the first columns :-/
d1 = dict(zip(tracks_cleaned.columns.levels[0], ['']))
d2 = dict(zip(tracks_cleaned.columns.levels[1], ['track_id']))

#foo = gla.rename(columns = {0: 'GLAAAA'}, level=0) # should work normally
tracks_cleaned = tracks_cleaned.rename(columns=d1, level=0)
tracks_cleaned = tracks_cleaned.rename(columns=d2, level=1)

tracks_cleaned.head(5)

{'Unnamed: 0_level_0': ''}

In [6]:
echonest.head(5)

Unnamed: 0_level_0           echonest                                        \
  Unnamed: 0_level_1     audio_features                                         
  Unnamed: 0_level_2       acousticness       danceability             energy   
            track_id Unnamed: 1_level_3 Unnamed: 2_level_3 Unnamed: 3_level_3   
0                  2           0.416675           0.675894           0.634476   
1                  3           0.374408           0.528643           0.817461   
2                  5           0.043567           0.745566           0.701470   
3                 10           0.951670           0.658179           0.924525   
4                134           0.452217           0.513238           0.560410   

                                                                               \
                                                                                
    instrumentalness           liveness        speechiness              tempo   
  Unnamed: 4_level_3 Unnamed: 5_level_3 Unnamed: 6_level_3 Unnamed: 7_level_3   
0           0.010628           0.177647           0.159310            165.922   
1           0.001851           0.105880           0.461818            126.957   
2           0.000697           0.373143           0.124595            100.260   
3           0.965427           0.115474           0.032985            111.562   
4           0.019443           0.096567           0.525519            114.290   

                                                ...           \
                               metadata         ...            
             valence         album_date         ...            
  Unnamed: 8_level_3 Unnamed: 9_level_3         ...            
0           0.576661                NaN         ...            
1           0.269240                NaN         ...            
2           0.621661                NaN         ...            
3           0.963590         2008-03-11         ...            
4           0.894072                NaN         ...            

                                                                  \
     temporal_features                                             
                   214                  215                  216   
  Unnamed: 240_level_3 Unnamed: 241_level_3 Unnamed: 242_level_3   
0            -1.992303             6.805694             0.233070   
1            -1.582331             8.889308             0.258464   
2            -2.288358            11.527109             0.256821   
3            -3.662988            21.508228             0.283352   
4            -1.452696             2.356398             0.234686   

                                                                  \
                                                                   
                   217                  218                  219   
  Unnamed: 243_level_3 Unnamed: 244_level_3 Unnamed: 245_level_3   
0             0.192880             0.027455              0.06408   
1             0.220905             0.081368              0.06413   
2             0.237820             0.060122              0.06014   
3             0.267070             0.125704              0.08082   
4             0.199550             0.149332              0.06440   

                                                                  \
                                                                   
                   220                  221                  222   
  Unnamed: 246_level_3 Unnamed: 247_level_3 Unnamed: 248_level_3   
0              3.67696              3.61288            13.316690   
1              6.08277              6.01864            16.673548   
2              5.92649              5.86635            16.013849   
3              8.41401              8.33319            21.317064   
4             11.26707             11.20267            26.454180   

                        
                        
                   223  
  Unnamed: 249_level_3  
0           262.929749  
1

In [246]:
# Copy & removing useless columns labels in level 2
echonest_cleaned = echonest.copy()
echonest_cleaned.columns = echonest_cleaned.columns.droplevel(level=3)

# TODO BEURKKK pas bien xD
# Need to create a dictionnary to rename the first columns :-/
d1 = dict(zip(echonest_cleaned.columns.levels[0], ['']))
d2 = dict(zip(echonest_cleaned.columns.levels[1], ['']))
d3 = dict(zip(echonest_cleaned.columns.levels[2], ['track_id']))

#foo = gla.rename(columns = {0: 'GLAAAA'}, level=0) # should work normally
echonest_cleaned = echonest_cleaned.rename(columns=d1, level=0)
echonest_cleaned = echonest_cleaned.rename(columns=d2, level=1)
echonest_cleaned = echonest_cleaned.rename(columns=d3, level=2) # BUGGG TODO


echonest_cleaned.head(5)

echonest                                          \
                     audio_features                                           
  Unnamed: 0_level_2   acousticness danceability    energy instrumentalness   
0                  2       0.416675     0.675894  0.634476         0.010628   
1                  3       0.374408     0.528643  0.817461         0.001851   
2                  5       0.043567     0.745566  0.701470         0.000697   
3                 10       0.951670     0.658179  0.924525         0.965427   
4                134       0.452217     0.513238  0.560410         0.019443   

                                                           ...      \
                                              metadata     ...       
   liveness speechiness    tempo   valence  album_date     ...       
0  0.177647    0.159310  165.922  0.576661         NaN     ...       
1  0.105880    0.461818  126.957  0.269240         NaN     ...       
2  0.373143    0.124595  100.260  0.621661         NaN     ...       
3  0.115474    0.032985  111.562  0.963590  2008-03-11     ...       
4  0.096567    0.525519  114.290  0.894072         NaN     ...       

                                                                       \
  temporal_features                                                     
                214        215       216       217       218      219   
0         -1.992303   6.805694  0.233070  0.192880  0.027455  0.06408   
1         -1.582331   8.889308  0.258464  0.220905  0.081368  0.06413   
2         -2.288358  11.527109  0.256821  0.237820  0.060122  0.06014   
3         -3.662988  21.508228  0.283352  0.267070  0.125704  0.08082   
4         -1.452696   2.356398  0.234686  0.199550  0.149332  0.06440   

                                              
                                              
        220       221        222         223  
0   3.67696   3.61288  13.316690  262.929749  
1   6.08277   6.01864  16.673548  325.581085  
2   5.92649   5.86635  16.013849  356.755737  
3   8.41401   8.33319  21.317064  483.403809  
4  11.26707  11.20267  26.454180  751.147705  

[5 rows x 250 columns]

In [7]:
features.head(5)

feature        chroma_cens                                        \
  statistics           kurtosis                                         
      number                 01                 02                 03   
    track_id Unnamed: 1_level_3 Unnamed: 2_level_3 Unnamed: 3_level_3   
0          2           7.180653           5.230309           0.249321   
1          3           1.888963           0.760539           0.345297   
2          5           0.527563          -0.077654          -0.279610   
3         10           3.702245          -0.291193           2.196742   
4         20          -0.193837          -0.198527           0.201546   

                                                                               \
                                                                                
                  04                 05                 06                 07   
  Unnamed: 4_level_3 Unnamed: 5_level_3 Unnamed: 6_level_3 Unnamed: 7_level_3   
0           1.347620           1.482478           0.531371           1.481593   
1           2.295201           1.654031           0.067592           1.366848   
2           0.685883           1.937570           0.880839          -0.923192   
3          -0.234449           1.367364           0.998411           1.770694   
4           0.258556           0.775204           0.084794          -0.289294   

                                                ...           \
                                                ...            
                  08                 09         ...            
  Unnamed: 8_level_3 Unnamed: 9_level_3         ...            
0           2.691455           0.866868         ...            
1           1.054094           0.108103         ...            
2          -0.927232           0.666617         ...            
3           1.604566           0.521217         ...            
4          -0.816410           0.043851         ...            

               tonnetz                                            \
                   std                                             
                    04                   05                   06   
  Unnamed: 509_level_3 Unnamed: 510_level_3 Unnamed: 511_level_3   
0             0.054125             0.012226             0.012111   
1             0.063831             0.014212             0.017740   
2             0.040730             0.012691             0.014759   
3             0.074358             0.017952             0.013921   
4             0.095003             0.022492             0.021355   

                   zcr                                            \
              kurtosis                  max                 mean   
                    01                   01                   01   
  Unnamed: 512_level_3 Unnamed: 513_level_3 Unnamed: 514_level_3   
0             5.758890             0.459473             0.085629   
1             2.824694             0.466309             0.084578   
2             6.808415             0.375000             0.053114   
3            21.434212             0.452148             0.077515   
4            16.669037             0.469727             0.047225   

                                                                  \
                median                  min                 skew   
                    01                   01                   01   
  Unnamed: 515_level_3 Unnamed: 516_level_3 Unnamed: 517_level_3   
0             0.071289             0.000000             2.089872   
1             0.063965             0.000000             1.716724   
2             0.041504             0.000000             2.193303   
3             0.071777             0.000000             3.542325   
4             0.040039             0.000977             3.189831   

                        
                   std  
                    01  
  Unnamed: 518_level_3  
0             0.061448  
1             0.069330  
2             0.044861  
3             0.040800  
4

In [247]:
# Copy & removing useless columns labels in level 2
features_cleaned = features.copy()
features_cleaned.columns = features_cleaned.columns.droplevel(level=3)

# TODO BEURKKK pas bien xD
# Need to create a dictionnary to rename the first columns :-/
d1 = dict(zip(features_cleaned.columns.levels[0], ['']))
d2 = dict(zip(features_cleaned.columns.levels[1], ['']))
d3 = dict(zip(features_cleaned.columns.levels[2], ['track_id']))

#foo = gla.rename(columns = {0: 'GLAAAA'}, level=0) # should work normally
features_cleaned = features_cleaned.rename(columns=d1, level=0)
features_cleaned = features_cleaned.rename(columns=d2, level=1)
features_cleaned = features_cleaned.rename(columns=d3, level=2) # BUGGG TODO


features_cleaned.head(5)

feature                                                              \
  statistics                                                               
      number  track_id        02        03        04        05        06   
0          2  7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
1          3  1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
2          5  0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
3         10  3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
4         20 -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   

                                   ...      tonnetz                      \
                                   ...          std                       
         07        08        09    ...           04        05        06   
0  1.481593  2.691455  0.866868    ...     0.054125  0.012226  0.012111   
1  1.366848  1.054094  0.108103    ...     0.063831  0.014212  0.017740   
2 -0.923192 -0.927232  0.666617    ...     0.040730  0.012691  0.014759   
3  1.770694  1.604566  0.521217    ...     0.074358  0.017952  0.013921   
4 -0.289294 -0.816410  0.043851    ...     0.095003  0.022492  0.021355   

         zcr                                                              
                   max      mean    median       min      skew       std  
    track_id  track_id  track_id  track_id  track_id  track_id  track_id  
0   5.758890  0.459473  0.085629  0.071289  0.000000  2.089872  0.061448  
1   2.824694  0.466309  0.084578  0.063965  0.000000  1.716724  0.069330  
2   6.808415  0.375000  0.053114  0.041504  0.000000  2.193303  0.044861  
3  21.434212  0.452148  0.077515  0.071777  0.000000  3.542325  0.040800  
4  16.669037  0.469727  0.047225  0.040039  0.000977  3.189831  0.030993  

[5 rows x 519 columns]

In [8]:
genres.head(5)

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5


In [9]:
# Get only the small subset tracks
tracks_small = cleaned_tracks[cleaned_tracks.subset == 'small']
tracks_small.sample(5)

NameError: name 'cleaned_tracks' is not defined

In [ ]:
# The main genres in the small data subset
tracks_small.genre_top.value_counts()
print('{} top-level genres'.format(len(genres['top_level'].unique())))
#genres.loc[genres['top_level'].unique()].sort_values('#tracks', ascending=False)

gla = genres.copy()#.groupby('top_level').agg({'#tracks': 'sort'})
gla.sort_values('#tracks', ascending=False).drop_duplicates(subset=['top_level'])

In [ ]:
genres.sort_values('#tracks').head(10)

In [ ]:
train = cleaned_tracks[cleaned_tracks['split'] == 'training']
val = cleaned_tracks[cleaned_tracks['split'] == 'validation']
test = cleaned_tracks[cleaned_tracks['split'] == 'test']
tracks

In [ ]:
features.head()

In [ ]:
features.iloc[0].unique()

In [ ]:
features.iloc[1].unique()

In [ ]:
features.columns.unique()

In [ ]:
features.feature.is_unique

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [ ]:
# Your code here
 
adjacency = # the adjacency matrix
n_nodes = # the number of nodes in the network

## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
plt.spy(adjacency, markersize=1)
plt.title('adjacency matrix')

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
# Your code here.

**Your answer here.**

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

**Your answer here.**

In [ ]:
# Your code here.

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
# Your code here.

### Question 4

Are the edges of your graph weighted?

**Your answer here.**

### Question 5

What is the degree distibution of your network? 

In [ ]:
degree =  # Your code here. It should be a numpy array.

assert len(degree) == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [ ]:
# Your code here.

### Question 6

Comment on the degree distribution of your network.

**Your answer here.**

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
def connected_graph(adjacency):
    """Determines whether a graph is connected.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """
    
    # Your code here.
    
    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
# Your code here.

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """
    
    # Your code here.
    
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [ ]:
# Your code here.

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [ ]:
def compute_shortest_path_lengths(adjacency, source):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """
    
    # Your code here.
    
    return shortest_path_lengths

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
# Your code here.

### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    
    # Your code here.
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
# Your code here.

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    # Your code here.
    
    return clustering_coefficient

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
# Your code here.